In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
load_dotenv();

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you do know about {country}",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({"country":"Korea"})

/var/folders/m7/5b9m5cbj3ls245_qrm3sq5l00000gp/T/ipykernel_83544/879333644.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


AI:
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi, Bibimbap, Bulgogi
        Currency: South Korean Won

AIMessage(content='AI:\n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi, Bibimbap, Bulgogi\n        Currency: South Korean Won', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-93f65893-5627-4ccc-9e36-6e2abf9fe8d7-0')